In [2]:
import NewTensor as nt
import pandas as pd
import math

In [3]:
mas = nt.Tensor([[1,2]])

In [8]:
import math

def Tanh(x):
    return math.tanh(x)
def ReLU(x, **kwargs):
    if x <= 0:
        return 0
    return kwargs.get('weight',1)*x

Derivative_list = {
    Tanh: lambda x, **kwargs: 2/(1 + math.e**(-2*x)) - 1,
    ReLU: lambda x, **kwargs: kwargs.get('weight', 1) if x >= 0 else 0
}
def Derivative_Activation(input, act, **kwargs):
    shape = input.shape()
    if len(shape) == 1:
        for i in range(shape[0]):
            res = Derivative_list[act](input[i], **kwargs)
            input[i] = res
        return
    for i in range(shape[0]):
        Derivative_Activation(input[i], act, **kwargs)
    return input

def Activation(input, act, **kwargs):
    shape = input.shape()
    if len(shape) == 1:
        for i in range(shape[0]):
            res = act(input[i])
            input[i] = res
        return
    for i in range(shape[0]):
        Activation(input[i], act, **kwargs)

    return input

TypeError: 'weight' is an invalid keyword argument for print()

[[[1, 2], [0, 0]], [[1, 2]]]

In [1]:
#def ReLu(input):
#    for i in range(len(input)):
#        for j in range(len(input[i])):
#            input[i][j] = max(input[i][j], 0)
#    return input
#def DevirativeReLU(input):
#    for i in range(len(input)):
#        for j in range(len(input[i])):
#            if input[i][j] > 0:
#                input[i][j] = 1
#            else:
#                input[i][j] = 0
#    return input
def MSE_loss_devirative(input, target):
    return 2*(input-target)
def MSE_loss(input, target):
    return (input-target)*(input-target)

In [ ]:
class Linear:
    def __init__(self, input_size, output_size,activation,devirative_activation,seed = None, **kwargs):
        self.input_size = input_size
        self.output_size = output_size
        self.weights = nt.rand_tensor([input_size+1,output_size],seed=seed)
        self.gradients = nt.zeros([input_size+1,output_size])
        self.activation = activation
        self.devirative_activation = Derivative_Activation(activation)
    def __call__(self, inputs):
        if len(inputs[0]) == self.input_size:
            for i in range(len(inputs)):
                inputs[i].append(1)
        self.input = inputs
        self.output = self.activation(nt.matmul(inputs, self.weights))
        return self.output
    def backward(self, error):
        error = self.devirative_activation(self.output, self.activation)*error
        self.gradients = nt.matmul(self.input.transpose(),error)
        return nt.matmul(error, self.weights.transpose())

In [ ]:
class NeuralNetwork:
    def __init__(self,sequence):
        self.sequence = sequence
        self.loss = MSE_loss
        self.devirative_loss = MSE_loss_devirative
    def __call__(self, inputs):
        for i in self.sequence:
            inputs = i(inputs)
        return inputs
    def gradient_backward(self,inputs,target):
        predict = self(inputs)
        self.error = self.devirative_loss(predict,target)
        for i in range(len(self.sequence)):
            self.error = self.sequence[len(self.sequence)-i-1].backward(self.error)
            for i in range(len(self.error)):
                self.error[i].pop(-1)
    def update_weights(self,lr = 0.001):
        for i in self.sequence:
            i.weights  = i.weights - lr * i.gradients
    def zero_gradient(self):
        for i in self.sequence:
            i.gradients = nt.zeros(i.gradients.shape())
    def fit(self,input,target,lr = 0.001,epochs = 1000):
        for epoch in range(epochs):
            self.gradient_backward(input,target)
            self.update_weights(lr)
            self.zero_gradient()
    def test(self):
        

In [ ]:
model = NeuralNetwork([Linear(1,1)])

In [ ]:
data = pd.read_csv('IRIS.csv',sep=',')
y_column = 'species'
types = list(data[y_column].unique())
types_to_ind = {specie:i for i, specie in enumerate(types)}
for i in range(len(data)):
    if data[y_column][i] in types:
        data.loc[i, y_column] = types_to_ind[data.loc[i, y_column]]
import random
def data_split(data,train_size=0.8):
    indexes = [i for i in range(len(data))]
    choices = random.sample(indexes,int(train_size*len(data)))
    train_x = [[data.iloc[i].iloc[j] for j in range(4)] for i in choices]
    train_y = [data.iloc[i].iloc[4] for i in choices]
    val_x = [[data.iloc[i].iloc[j] for j in range(4)] for i in indexes if i not in choices]
    val_y = [data.iloc[i].iloc[4] for i in indexes if i not in choices]
    return train_x,train_y,val_x,val_y
train_x,train_y,val_x,val_y = data_split(data)
train_x = nt.Tensor(train_x).float()
train_y = nt.Tensor(train_y).float()
val_x = nt.Tensor(val_x).float()
val_y = nt.Tensor(val_y).float()
